In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\python\\Deep Learning Project\\Kidney-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
        )
        
        create_directories([Path(training.root_dir)])

        

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )

        return training_config
    



In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
# Check if GPU is available
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("GPU available")
    # Additional information about the GPUs
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU available")

GPU available
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.gpu_available = tf.config.list_physical_devices("GPU")

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(rescale=1.0 / 255, validation_split=0.20)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train_model(self):
        self.steps_per_epoch = (
            self.train_generator.samples // self.train_generator.batch_size
        )
        self.validation_steps = (
            self.valid_generator.samples // self.valid_generator.batch_size
        )

        start_time = time.time()
        for epoch in range(self.config.params_epochs):
            print(f"Epoch {epoch + 1}/{self.config.params_epochs}")

            # Training loop
            epoch_start_time = time.time()
            self.model.fit(
                self.train_generator,
                steps_per_epoch=self.steps_per_epoch,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator,
                verbose=2
            )
            epoch_end_time = time.time()

            epoch_time = epoch_end_time - epoch_start_time
            print(f"Epoch {epoch + 1} took {epoch_time:.2f} seconds")

        total_time = time.time() - start_time
        print(f"Training completed in {total_time:.2f} seconds")

        
    def train(self):
        if self.gpu_available:
            print("GPU available. Using GPU...")
            with tf.device("/GPU:0"):
                self.train_model()
        else:
            print("No GPU available. Using CPU...")
            self.train_model()

        self.save_model(path=self.config.trained_model_path, model=self.model)

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
GPU available. Using GPU...
Epoch 1/5
155/155 - 297s - loss: 1.3826 - accuracy: 0.3990 - val_loss: 1.1867 - val_accuracy: 0.5633 - 297s/epoch - 2s/step
Epoch 1 took 298.80 seconds
Epoch 2/5
155/155 - 149s - loss: 1.1250 - accuracy: 0.5267 - val_loss: 1.0964 - val_accuracy: 0.5461 - 149s/epoch - 958ms/step
Epoch 2 took 149.46 seconds
Epoch 3/5
155/155 - 161s - loss: 0.9967 - accuracy: 0.5994 - val_loss: 1.1351 - val_accuracy: 0.6414 - 161s/epoch - 1s/step
Epoch 3 took 161.78 seconds
Epoch 4/5
155/155 - 143s - loss: 0.9193 - accuracy: 0.6410 - val_loss: 1.0588 - val_accuracy: 0.5855 - 143s/epoch - 921ms/step
Epoch 4 took 143.59 seconds
Epoch 5/5
155/155 - 153s - loss: 0.8780 - accuracy: 0.6574 - val_loss: 1.0614 - val_accuracy: 0.6040 - 153s/epoch - 987ms/step
Epoch 5 took 153.97 seconds
Training completed in 907.60 seconds
